In [1]:
# Load json from data files

In [18]:
#Load Tweets
import json
import pandas as pd

def read_input(input_txt_file):
    tweets_list = []
    #print("Start Reading Text file containing multiple JSON objects -> Return Python dictionary (key-value) of JSON objets")
    with open(input_txt_file) as file:
        for jsonObject in file:
            if not jsonObject.isspace():
                tweets_Dict = json.loads(jsonObject)
                tweets_list.append(tweets_Dict)

    tweet_data_frame = pd.DataFrame.from_dict(tweets_list)
    #filter out retweeted tweets
    tweet_data_frame=tweet_data_frame[tweet_data_frame.retweeted==False]
    # Lowercase --> column TEXT
    tweet_data_frame["text"] = tweet_data_frame["text"].apply(lambda str: str.lower())
   
    return tweet_data_frame


In [19]:
!pwd

/home/ubuntu/lab3


In [20]:
input_path = "./data/05cb5036-2170-401b-947d-68f9191b21c6"

In [21]:
tweet_data_frame = read_input(input_path)
tweet_data_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6690 entries, 0 to 6689
Data columns (total 31 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   created_at                 6690 non-null   object 
 1   id                         6690 non-null   int64  
 2   id_str                     6690 non-null   object 
 3   text                       6690 non-null   object 
 4   source                     6690 non-null   object 
 5   truncated                  6690 non-null   bool   
 6   in_reply_to_status_id      2516 non-null   float64
 7   in_reply_to_status_id_str  2516 non-null   object 
 8   in_reply_to_user_id        2574 non-null   float64
 9   in_reply_to_user_id_str    2574 non-null   object 
 10  in_reply_to_screen_name    2574 non-null   object 
 11  user                       6690 non-null   object 
 12  geo                        13 non-null     object 
 13  coordinates                13 non-null     objec

In [28]:
texts = tweet_data_frame['text']
texts

0         @danielsbetting ska fan radera den appen alltså
1                         @vinylprinsessa hen heter länis
2       @kortvilse ja se folk är aldrig nöjda... 😁 \nd...
3       @nyfikenful hahahaha.... jädrans bra svar! 👍✌️...
4       rt @vrakplundrare: ni som är oroliga för usa o...
                              ...                        
6685    rt @nabila_afr: jag älskar att #lyckligagatan ...
6686              @eddiescurfy den twitter support? ;) :p
6687    rt @bodinhenrik: att följa någon på twitter oc...
6688    det enda glädjeämnet jag finner är att molins ...
6689    @silvermoback jag har hejat på adde men han jo...
Name: text, Length: 6690, dtype: object

In [30]:
import re
hen_patter = re.compile("[^a-z]hen[^a-z]")
han_patter = re.compile("[^a-z]han[^a-z]")
hon_patter = re.compile("[^a-z]hon[^a-z]")
den_patter = re.compile("[^a-z]den[^a-z]")
det_patter = re.compile("[^a-z]det[^a-z]")
denna_patter = re.compile("[^a-z]denna[^a-z]")
denne_patter = re.compile("[^a-z]denne[^a-z]")
counters = {'hen':0,'han':0,'hon':0,'den':0,'det':0,'denna':0,'denne':0}

In [34]:
for index, line in texts.items():
     # Hen Patter     
    if hen_patter.search(line):
        counters['hen'] += 1
    # “han” Patter       
    if han_patter.search(line):
        counters['han'] += 1
    # HON        
    if hon_patter.search(line):
         counters['hon'] += 1
           # DET        
    if den_patter.search(line):
        counters['den'] += 1
        # DET        
    if det_patter.search(line):
         counters['det'] += 1
    # DENNA        
    if denna_patter.search(line):
         counters['denna'] += 1
    # DENNE       
    if denne_patter.search(line):
        counters['denne'] += 1

In [35]:
counters

{'hen': 128,
 'han': 4152,
 'hon': 874,
 'den': 6858,
 'det': 2584,
 'denna': 98,
 'denne': 54}

In [7]:
# Celery tasks test
from proj.tasks import add, mul

In [8]:

add.delay(4, 4)

<AsyncResult: 5d01bf90-efba-421b-8448-5357305369a9>

In [9]:
res = mul.delay(4, 4)
print(res.ready())
res.get(timeout=1)

True


16